In [ ]:
# import the necessary packages
from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os
from os import listdir
from os.path import isfile, join
from pathlib import Path

import tkinter as tk

# import the necessary packages
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

from imutils.video import VideoStream
from imutils.video import FPS
import time

In [ ]:
#hit space bar to capture
def captureuser(name):
    cam = cv2.VideoCapture(0)

    cv2.namedWindow("capture")

    img_counter = 0
    
    dirname = f'dataset/{name}'
    os.mkdir(dirname)

    while True:
        ret, frame = cam.read()
        cv2.imshow("capture", frame)
        
        if img_counter == 5:
            cv2.destroyWindow("capture")
            break
        if not ret:
            break
        k = cv2.waitKey(1)

        if k%256 == 27:
            # ESC pressed
            print("Escape hit, closing...")
            break
        elif k%256 == 32:
            # SPACE pressed
            path = f'dataset/{name}'
            img_name = "{}.jpg".format(img_counter)
            cv2.imwrite(os.path.join(path , img_name), frame)
            cv2.imwrite(img_name, frame)
            print("{} written!".format(img_name))
            img_counter += 1

    cam.release()

    cv2.destroyAllWindows()

In [ ]:
captureuser('user')

In [ ]:
def write_to_csv(unique_id,name,bank, password, account_balance):
    import csv
    
    with open(r'bank_details.csv','a', newline = '\n') as f:
        writer = csv.writer(f)
        writer.writerow([unique_id,name,bank, password, account_balance])

In [ ]:
def get_embeddings():
    #summary:
    
    ap = argparse.ArgumentParser()
    ap.add_argument("-i", "--dataset", required=True,
        help="path to input directory of faces + images")
    ap.add_argument("-e", "--embeddings", required=True,
        help="path to output serialized db of facial embeddings")
    ap.add_argument("-d", "--detector", required=True,
        help="path to OpenCV's deep learning face detector")
    ap.add_argument("-m", "--embedding-model", required=True,
        help="path to OpenCV's deep learning face embedding model")
    ap.add_argument("-c", "--confidence", type=float, default=0.5,
        help="minimum probability to filter weak detections")
    
    print("[INFO] loading face detector...")

    detector = cv2.dnn.readNetFromCaffe('face_detection_model/deploy.prototxt', 'face_detection_model/res10_300x300_ssd_iter_140000.caffemodel')
    embedder = cv2.dnn.readNetFromTorch('openface_nn4.small2.v1.t7')

    print("[INFO] quantifying faces...")
    imagePaths = list(paths.list_images('dataset'))
    knownEmbeddings = []
    knownNames = []
    total = 0
    for (i, imagePath) in enumerate(imagePaths):
        print("[INFO] processing image {}/{}".format(i + 1,
            len(imagePaths)))
        name = imagePath.split(os.path.sep)[-2]

        image = cv2.imread(imagePath)
        image = imutils.resize(image, width=600)
        (h, w) = image.shape[:2]
        imageBlob = cv2.dnn.blobFromImage(
            cv2.resize(image, (300, 300)), 1.0, (300, 300),
            (104.0, 177.0, 123.0), swapRB=False, crop=False)

        detector.setInput(imageBlob)
        detections = detector.forward()

        if len(detections) > 0:
            
            i = np.argmax(detections[0, 0, :, 2])
            confidence = detections[0, 0, i, 2]

           
            if confidence > 0.5:
                
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                face = image[startY:endY, startX:endX]
                (fH, fW) = face.shape[:2]

                if fW < 20 or fH < 20:
                    continue

                faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
                    (96, 96), (0, 0, 0), swapRB=True, crop=False)
                embedder.setInput(faceBlob)
                vec = embedder.forward()
    
                knownNames.append(name)
                knownEmbeddings.append(vec.flatten())
                total += 1
    print("[INFO] serializing {} encodings...".format(total))
    data = {"embeddings": knownEmbeddings, "names": knownNames}
    f = open('output/embeddings.pickle', "wb")
    f.write(pickle.dumps(data))
    f.close()


In [ ]:
get_embeddings()

In [ ]:
def train_model():
    #summary    
    
    print("[INFO] loading face embeddings...")
    data = pickle.loads(open('output/embeddings.pickle', "rb").read())
    le = LabelEncoder()
    labels = le.fit_transform(data["names"])
    
    print("[INFO] training model...")
    recognizer = SVC(C=1.0, kernel="linear", probability=True)
    recognizer.fit(data["embeddings"], labels)
    f = open('output/recognizer.pickle', "wb")
    f.write(pickle.dumps(recognizer))
    f.close()

    f = open('output/le.pickle', "wb")
    f.write(pickle.dumps(le))
    f.close()

In [ ]:
train_model()